# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [46]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [47]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gabane,-24.6667,25.7822,22.29,72,92,1.69,BW,1735604830
1,1,mayo-belwa,9.0500,12.0500,22.46,24,70,0.99,NG,1735604832
2,2,grytviken,-54.2811,-36.5092,5.85,79,54,3.07,GS,1735604833
3,3,ushuaia,-54.8000,-68.3000,8.81,100,100,1.54,AR,1735604588
4,4,hamilton,39.1834,-84.5333,8.90,78,0,3.09,US,1735604406


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [48]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng', 
    'Lat',
    geo = True,
    frame_width = 800,
    frame_height = 600,
    tiles = 'OSM',
    size = 'Humidity',
    alpha = 0.75,
    scale = 1,
    color = 'City'
    )

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[
    (city_data_df['Max Temp'] >= 17) & (city_data_df['Max Temp'] < 24) &
    (city_data_df['Wind Speed'] < 4.25) &
    (city_data_df['Cloudiness'] <= 1) &
    (city_data_df['Country'] != 'US')
    ]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,charcas,23.1333,-101.1167,18.30,16,0,3.05,MX,1735604842
25,25,margaret river,-33.9500,115.0667,19.32,50,1,3.75,AU,1735604862
107,107,gorom-gorom,14.4439,-0.2361,22.88,13,0,3.59,BF,1735604960
140,140,seeb,23.6703,58.1891,19.05,72,0,1.54,OM,1735604999
202,202,pong,19.1493,100.2752,19.11,74,0,0.44,TH,1735605079
210,210,eldorado,-23.7869,-54.2836,22.89,60,0,2.20,BR,1735605058
212,212,huambo,-12.7761,15.7392,17.89,88,0,3.09,AO,1735605093
218,218,konarka,19.9000,86.1167,19.61,73,0,3.34,IN,1735605100
221,221,kabala,9.5833,-11.5500,19.96,22,0,1.90,SL,1735605103
228,228,kidal,18.4411,1.4078,20.10,13,0,4.01,ML,1735605111


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = (city_data_df.copy())[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,charcas,MX,23.1333,-101.1167,16,
25,margaret river,AU,-33.9500,115.0667,50,
107,gorom-gorom,BF,14.4439,-0.2361,13,
140,seeb,OM,23.6703,58.1891,72,
202,pong,TH,19.1493,100.2752,74,
210,eldorado,BR,-23.7869,-54.2836,60,
212,huambo,AO,-12.7761,15.7392,88,
218,konarka,IN,19.9000,86.1167,73,
221,kabala,SL,9.5833,-11.5500,22,
228,kidal,ML,18.4411,1.4078,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
charcas - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
gorom-gorom - nearest hotel: No hotel found
seeb - nearest hotel: فندق جولدن كراون السيب
pong - nearest hotel: No hotel found
eldorado - nearest hotel: Hotel Laçador
huambo - nearest hotel: No hotel found
konarka - nearest hotel: Hotel Sun Village
kabala - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
betioky - nearest hotel: Hotel Salema
port lincoln - nearest hotel: Boston Hotel
general jose de san martin - nearest hotel: Hotel Belgrado
porbandar - nearest hotel: Toran Tourist Bungalow
pisco - nearest hotel: La Portada
marsh harbour - nearest hotel: Lofty Fig Villas
doctor arroyo - nearest hotel: No hotel found
malvan - nearest hotel: Hotel Silver Sand


,City,Country,Lat,Lng,Humidity,Hotel Name
9,charcas,MX,23.1333,-101.1167,16,No hotel found
25,margaret river,AU,-33.9500,115.0667,50,Margaret River Hotel
107,gorom-gorom,BF,14.4439,-0.2361,13,No hotel found
140,seeb,OM,23.6703,58.1891,72,فندق جولدن كراون السيب
202,pong,TH,19.1493,100.2752,74,No hotel found
210,eldorado,BR,-23.7869,-54.2836,60,Hotel Laçador
212,huambo,AO,-12.7761,15.7392,88,No hotel found
218,konarka,IN,19.9000,86.1167,73,Hotel Sun Village
221,kabala,SL,9.5833,-11.5500,22,No hotel found
228,kidal,ML,18.4411,1.4078,13,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != 'No hotel found', :]
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    color = 'City',
    size = 'Humidity',
    scale = 2.25,
    frame_width = 800,
    frame_height = 600,
    alpha = 0.8,
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)